# **2110443 - Computer Vision (2022/2)**
#**Lab 9 - Modern Object Detection [Homework]** <br>
In this lab, we will learn how to use MMDetection to perform object detection in the provided real world dataset using PyTorch. This notebook includes both coding and written questions. Please hand in this notebook file with all outputs and your answer.

**Collaboration is encouraged in this course.** You must turn in your own write ups of all problems. If you collaborate with others, you must put the names and ids of the students you worked with in below block.

Collaboration List:
- ...
- ...


# Assignment 1 : Parasite Detection
![alt text](https://icip2022challenge.piclab.ai/wp-content/uploads/2022/01/parasite_egg_11_with_size.png) 

In this assignment you have to replace YOUR_STUDENT_ID_WITH21 variable with your student id (in integer). There will be 2 sets of data: train and test 

By using the knowledge from the lab and lecture, you have to train object detection model and tested on unknown label dataset!



Scoreboard URL : https://www.piclab.ai/classes/cv2022/lab9/scoreboard

Install prerequisite libraries for MMDetection


In [ ]:
!pip install openmim
!mim install mmdet

In [ ]:
import random
import glob
import os
import numpy as np
import cv2
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import matplotlib
from matplotlib import font_manager

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision import models as models
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


##### Don't forget to put your ID here (in integer) ####
YOUR_STUDENT_ID_WITH21 = 5630008021 
#######################################################

def isStudentIDValid(studentID):
  strID = str(studentID)
  isEndWith21 = strID.endswith('21')
  isLengthOK = len(strID) == 10
  if isEndWith21 and isLengthOK:
    return True
  else:
    return False

print('Student ID status:',isStudentIDValid(YOUR_STUDENT_ID_WITH21))

## Your model description goes here: ###
WRITE HERE

## GPU Status Check

In [ ]:
!nvidia-smi

## Download and inspect parasite dataset

In [ ]:
!pip install --upgrade gdown 
!gdown https://drive.google.com/uc?id=1RVluBzZvJ8mWwA5AuB2PklbaoK60uPMF
!gdown https://drive.google.com/uc?id=1kvBxmC1o0vE2ef2CJqhLaj9V4TD72oW3
!unzip parasite_train.zip
!unzip parasite_test.zip

### Download Model


In [ ]:
#### FILL YOUR CODE HERE ####


### Dataset Exploration
We will use pycocotools to explore this dataset. 

In [ ]:
from mmcv import Config

modelConfig = ???
print(f'Original Config:\n{modelConfig.pretty_text}')

In [ ]:
from pycocotools.coco import COCO
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
trainLabelFile='parasite_train/labels.json'
# initialize COCO api for instance annotations
trainCOCOBinding = COCO(trainLabelFile)

In [ ]:
#display COCO categories and supercategories
cats = trainCOCOBinding.loadCats(trainCOCOBinding.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

# get all images containing given categories, select one at random
catIds = trainCOCOBinding.getCatIds(catNms=['Hymenolepis diminuta',]);
imgIds = trainCOCOBinding.getImgIds(catIds=catIds );

randomImgId = np.random.randint(0,len(imgIds))
sampleImageData = trainCOCOBinding.loadImgs(randomImgId)[0]

print('Image Data >>', sampleImageData)

sampleImage = cv2.imread('parasite_train/data/'+sampleImageData['file_name'])

annIds = trainCOCOBinding.getAnnIds(imgIds=randomImgId, iscrowd=None)
boxes = trainCOCOBinding.loadAnns(annIds)
print('Box Data', boxes)

for box in boxes:
  x,y,w,h = box['bbox']
  cv2.rectangle(sampleImage, (int(x), int(y)), (int(x+w), int(y+h)), (0,255,0), 5)

sampleImage = cv2.cvtColor(sampleImage, cv2.COLOR_BGR2RGB)

plt.imshow(sampleImage)
plt.show()

### Modify MMDetection model configuration

In [ ]:
from mmcv.utils.config import ConfigDict
from mmdet.apis import set_random_seed

# Modify dataset type and path
#### FILL YOUR CODE HERE ####



print(f'Modified Config:\n{modelConfig.pretty_text}')

### Training

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import mmcv
import os

# Build dataset
datasets = [build_dataset(modelConfig.data.train)]

# Build the detecmodelConfig, distributed=False, validate=True)tor
model = build_detector(modelConfig.model, train_cfg=modelConfig.get('train_cfg'), test_cfg=modelConfig.get('test_cfg'))

# Create work_dir
mmcv.mkdir_or_exist(os.path.abspath(modelConfig.work_dir))
train_detector(model, datasets, 

### Inference on test set!

In [ ]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

In [ ]:
def get_coco_result(bboxes, id, score_thr, file_name):
    coco_results = []
    for i in range(len(bboxes)):
        # print(bboxes[i])
        if len(bboxes[i])> 0 and bboxes[i][:, 4].max() > score_thr:
            x1, y1, x2, y2 = bboxes[i][0, :4].tolist()
            width = x2 - x1
            height = y2 - y1
            x = x1
            y = y1
            score = float(bboxes[i][0, 4])
            coco_result = {
                'id': id,
                'category_id': i,  # COCO dataset class IDs start from 1
                'bbox': [x, y, width, height],
                'score': score,
                'file_name': file_name
            }
            
         
            id += 1
            coco_results.append(coco_result)
    return coco_results, id

In [ ]:
model = ???

### Submit Results

In [ ]:
from mmdet.core import  bbox2result

image_dir = '/content/parasite_test/data'

annotations = dict()
annotations['annotations'] = list()
id = 0
for filename in os.listdir(image_dir):
    if filename.endswith('.jpg'):
        image_path = os.path.join(image_dir, filename)
        
        result = inference_detector(model, image_path)
        # print(result)
        coco_result, id = get_coco_result(result, id, 0.3,filename)
        annotations['annotations']+=coco_result
        # break

In [ ]:
import requests
sendDict = { 'studentID':YOUR_STUDENT_ID_WITH21, 'results':  annotations }
response = requests.post('https://www.piclab.ai/classes/cv2022/lab9/scoreboard/submitLab9',headers={'Content-Type': 'application/json' }, json=sendDict)
print(response.text)